# Congruence Engine: Museums' online catalogue-as-data investigation
https://www.sciencemuseumgroup.org.uk/project/the-congruence-engine/

GitHub: https://github.com/congruence-engine/catalogues-as-data

In [ ]:
# Create a folder named 'data' to which to download, and later save, data:
!mkdir "./data"

In [1]:
# Download Science Museum Group's (SMG) objects dataset
# Source: https://github.com/congruence-engine/datasets

# Fetch and save the dataset:
!wget "https://github.com/congruence-engine/datasets/raw/main/smg/smg_objects_06_06_2022.json.zip" -O "./data/smg_objects_06_06_2022.json.zip"

# Unzip the downloaded dataset in the data directory:
!unzip -o "./data/smg_objects_06_06_2022.json.zip" -d "./data"

# The downloaded dataset should be found in the 'data' folder:
# .
# ├── data
# │ ├── smg_objects_06_06_2022.json         <-- SMG's dataset unzipped.
# │ └── smg_objects_06_06_2022.json.zip     <-- SMG's dataset downloaded.
# └── smg_objects.ipynb                     <-- This Notebook.

--2023-06-17 21:34:25--  https://github.com/congruence-engine/datasets/raw/main/smg/smg_objects_06_06_2022.json.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/congruence-engine/datasets/main/smg/smg_objects_06_06_2022.json.zip [following]
--2023-06-17 21:34:25--  https://media.githubusercontent.com/media/congruence-engine/datasets/main/smg/smg_objects_06_06_2022.json.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179577111 (171M) [application/zip]
Saving to: ‘./data/smg_objects_06_06_2022.json.zip’

./data/smg_objects_ 100%[===================>] 171.26M  5.91MB/s    in 

In [2]:
# Extract the required fields from the downloaded objects dataset
#
# The fields of interest (as identified by Anna-Maria, see SMG dataset mapping.xlsx):
# * _id
# * _source.title
# * _source.summary_title
# * _source.description
# * _source.name
# * _source.categories
# * _source.materials
# * _source.lifecycle.creation.date
#  
# The dataset is provided as JSON Lines (instead of a single JSON object, contains multiple JSON objects delimited by newline),
# and for ease of loading into Pandas dataframe, we'll convert the dataset to a regular JSON.
#
# All of this can be done in Python, but for convenience, we're using 'jq', a command line JSON processor.
# * jq (https://stedolan.github.io/jq/)
# * Give jq a try at jqplay (https://jqplay.org/)

# Extract fields:
!jq -r ". | {id: ._id, title: ._source.title, summary_title: ._source.summary_title, description: ._source.description, name: ._source.name, categories: ._source.categories, materials: ._source.materials, creation_date: ((._source.lifecycle.creation[]? | .date) // null)}" "./data/smg_objects_06_06_2022.json" > "./data/smg_objects_06_06_2022_extracted.json"

# Convert to regular JSON:
!jq -s . "./data/smg_objects_06_06_2022_extracted.json" > "./data/smg_objects_06_06_2022_extracted_converted.json"

# This results in the following JSON:
# [
#   {
#     "id": "co8714252",
#     "title": [
#       {
#         "primary": true,
#         "value": "Collection of veterinary operating sundries"
#       }
#     ],
#     "summary_title": "Collection of veterinary operating sundries (veterinary medicine)",
#     "description": [
#       {
#         "primary": true,
#         "value": "Collection of veterinary operating sundries, from the veterinary surgery of the Greyhound Racing Association, Northaw, 1950-1985."
#       }
#     ],
#     "name": [
#       {
#         "primary": true,
#         "value": "veterinary medicine"
#       }
#     ],
#     "categories": [
#       {
#         "museum": "SCM",
#         "name": "Veterinary Medicine",
#         "value": "SCM - Veterinary Medicine"
#       }
#     ],
#     "materials": null,
#     "creation_date": null
#   },
#   ...
# ]
#
# The extracted data should be found in the 'data' folder:
# .
# ├── data
# │ ├── smg_objects_06_06_2022.json
# │ ├── smg_objects_06_06_2022.json.zip
# │ ├── smg_objects_06_06_2022_extracted.json               <-- Data extracted from the SMG's dataset (JSONL).
# │ └── smg_objects_06_06_2022_extracted_converted.json     <-- Data extracted from the SMG's dataset (JSON).
# └── smg_objects.ipynb                                     <-- This Notebook.

In [3]:
# Import modules

import json
import pandas as pd

# Pandas dataframe display settings
pd.set_option('display.max_columns', None)

In [4]:
# Helper functions

def sort_columns(df):
    columns = df.columns.values.tolist()
    columns.remove('id')
    columns.sort()
    columns = ['id'] + columns
    df = df[columns]
    return df

In [5]:
# Load SMG's objects dataset

# Load the JSON file: 
with open('./data/smg_objects_06_06_2022_extracted_converted.json') as f:
    json_data = json.load(f)

In [6]:
# Load into dataframe: title (_source.title), summary_title (_source.summary_title)

# Load the field data as normalized (nested fields flattened):
df_title = pd.json_normalize(
    json_data,
    record_path =['title'],
    meta = ['id', 'summary_title'],
    record_prefix = 'title.'
)

# Tidy up:
df_title = df_title.convert_dtypes()
df_title = sort_columns(df_title)

# Remap title note:
df_title['title.note'] = df_title['title.note'].apply(lambda x: x[0]['value'] if not pd.isna(x) else x)
df_title.rename(columns={'title.note': 'title.note.value'}, inplace=True)

# Save to CSV file:
df_title.to_csv(
    './data/smg_objects_title.csv',
    index = False
)

df_title

,id,summary_title,title.note.value,title.primary,title.type,title.value
0,co8714252,Collection of veterinary operating sundries (v...,NaN,True,<NA>,Collection of veterinary operating sundries
1,co8413501,BBC Radio Manchester Audiopak (audio cassette),NaN,True,display title,BBC Radio Manchester Audiopak
2,co227371,Bath (painting; watercolour; poster artwork),NaN,True,display title,Bath
3,co8084133,Exacta single lens reflex camera (single lens ...,NaN,True,display title,exacta single lens reflex camera
4,co6308,Kelvin-White portable volt and amperemeter,NaN,True,display title,Kelvin-White portable volt and amperemeter
...,...,...,...,...,...,...
317120,co8538881,Mr Robot (robot; toy - recreational artefact),NaN,True,display title,Mr Robot
317121,co44908,Model (scale 1:6) of hand warping mill for sma...,NaN,True,display title,Model (scale 1:6) of hand warping mill for sma...
317122,co135801,"Iron model of the joints in a human skeleton, ...",NaN,True,S&H_TITLE,"Iron model of the joints in a human skeleton, ..."
317123,co8191358,Gramophone Record Turntable Transcription Unit...,NaN,True,<NA>,Gramophone Record Turntable Transcription Unit...


In [7]:
# Load into dataframe: description (_source.description)

# Load the field data as normalized (nested fields flattened):
df_description = pd.json_normalize(
    json_data,
    record_path =['description'],
    meta = ['id'],
    record_prefix = 'description.'
)

# Tidy up:
df_description = df_description.convert_dtypes()
df_description = sort_columns(df_description)

# Remap description dates:
df_description['description.date.earliest'] = df_description['description.date'].map(lambda x: pd.Period(x[0]['earliest']) if (type(x) == list and 'earliest' in x[0]) else x)
df_description['description.date.latest'] = df_description['description.date'].map(lambda x: pd.Period(x[0]['latest']) if (type(x) == list and 'latest' in x[0]) else x)
df_description['description.date.value'] = df_description['description.date'].map(lambda x: pd.Period(x[0]['value']) if (type(x) == list and 'value' in x[0]) else x)
df_description['description.date.primary'] = df_description['description.date'].map(lambda x: x[0]['primary'] if (type(x) == list and 'primary' in x[0]) else x)
df_description.drop(columns=['description.date'], inplace=True)

# Save to CSV file:
df_description.to_csv(
    './data/smg_objects_description.csv',
    index = False
)

df_description

,id,description.date.earliest,description.date.latest,description.date.value,description.primary,description.sort,description.source,description.type,description.value,description.date.primary
0,co8714252,NaT,NaT,NaN,True,<NA>,<NA>,<NA>,"Collection of veterinary operating sundries, f...",NaN
1,co8413501,2017,2017,2017-04-07,True,<NA>,<NA>,description,Broadcast cartridge used by Radio Manchester t...,True
2,co227371,NaT,NaT,NaN,True,<NA>,<NA>,<NA>,"Painting, watercolour, original artwork for ra...",NaN
3,co8084133,2011,2011,2011-03-28,True,<NA>,<NA>,description,Exacta camera. Single lens reflex camera for ...,True
4,co6308,NaT,NaT,NaN,True,<NA>,<NA>,accession register,Kelvin-White portable volt and amperemeter,NaN
...,...,...,...,...,...,...,...,...,...,...
376054,co135801,NaT,NaT,NaN,True,<NA>,<NA>,accession register,"Articulated manikin, iron, used to teach bone-...",NaN
376055,co135801,2008,2008,2008-11-19,<NA>,<NA>,<NA>,S&H web project text,The iron model showing the joints of a skeleto...,True
376056,co135801,NaT,NaT,NaN,<NA>,<NA>,<NA>,web description,The iron model showing the joints of a skeleto...,NaN
376057,co8191358,2011,2011,2011-12-23,True,<NA>,<NA>,description,Model 4T200 gramophone record turntable transc...,True


In [8]:
# Load into dataframe: name (_source.name)

# Load the field data as normalized (nested fields flattened):
df_name = pd.json_normalize(
    json_data,
    record_path =['name'],
    meta = ['id'],
    record_prefix = 'name.'
)

# Tidy up:
df_name = df_name.convert_dtypes()
df_name = sort_columns(df_name)

# Save to CSV file:
df_name.to_csv(
    './data/smg_objects_name.csv',
    index = False
)

df_name

,id,name.primary,name.type,name.value
0,co8714252,True,<NA>,veterinary medicine
1,co8413501,True,<NA>,audio cassette
2,co227371,<NA>,name,watercolour
3,co8084133,True,<NA>,single lens reflex camera
4,co442055,True,object name,plate
...,...,...,...,...
592475,co8538881,<NA>,name,toy - recreational artefact
592476,co44908,True,object type,warp mill
592477,co135801,True,<NA>,manikin
592478,co8191358,True,<NA>,gramophone


In [9]:
# Load field(s) into dataframe: categories (_source.categories)

# Load the field data as normalized (nested fields flattened):
df_categories = pd.json_normalize(
    json_data,
    record_path =['categories'],
    meta = ['id'],
    record_prefix = 'category.'
)

# Tidy up:
df_categories = df_categories.convert_dtypes()
df_categories = sort_columns(df_categories)

# Save to CSV file:
df_categories.to_csv(
    './data/smg_objects_categories.csv',
    index = False
)

df_categories

,id,category.museum,category.name,category.value
0,co8714252,SCM,Veterinary Medicine,SCM - Veterinary Medicine
1,co8413501,SIM,Radio Communication,SIM - Radio Communication
2,co227371,NRM,Pictorial Collection (Railway),NRM - Pictorial Collection (Railway)
3,co8084133,NSMM,Photographic Technology,NSMM - Photographic Technology
4,co6308,SCM,Electricity and Magnetism,SCM - Electricity and Magnetism
...,...,...,...,...
317153,co8538881,SCM,Human Robotics,SCM - Human Robotics
317154,co44908,SCM,Textiles Machinery,SCM - Textiles Machinery
317155,co135801,SCM,Therapeutics,SCM - Therapeutics
317156,co8191358,SCM,Sound Reproduction,SCM - Sound Reproduction


In [10]:
# Load field(s) into dataframe: materials (_source.materials)

# Load the field data as normalized (nested fields flattened):
df_materials = pd.json_normalize(
    json_data,
    record_path =['materials'],
    meta = ['id'],
    record_prefix = 'material'
)

# Tidy up:
df_materials = df_materials.convert_dtypes()
df_materials.rename(columns={'material0': 'material'}, inplace=True)
df_materials = sort_columns(df_materials)

# Save to CSV file:
df_materials.to_csv(
    './data/smg_objects_materials.csv',
    index = False
)

df_materials

,id,material
0,co8413501,metal (unknown)
1,co8413501,plastic (unidentified)
2,co8084133,glass
3,co8084133,leatherette
4,co8084133,metal (unknown)
...,...,...
256768,co8191358,paint
256769,co8191358,plastic (unidentified)
256770,co8191358,rubber (unidentified)
256771,co8191358,steel (metal)


In [11]:
# Load field(s) into dataframe: creation_date (_source.lifecycle.creation.date)

# Load the field data as normalized (nested fields flattened):
df_creation_date = pd.json_normalize(
    json_data,
    record_path =['creation_date'],
    meta = ['id'],
    record_prefix = 'creation.date.'
)

# Tidy up:
df_creation_date = df_creation_date.convert_dtypes()
df_creation_date = sort_columns(df_creation_date)

# Save to CSV file:
df_creation_date.to_csv(
    './data/smg_objects_lifecycle_creation_date.csv',
    index = False
)

df_creation_date

,id,creation.date.earliest,creation.date.latest,creation.date.primary,creation.date.value
0,co227371,1956,1956,True,1956
1,co8084133,1934,1934,True,1934
2,co6308,1906,1906,True,1906
3,co442055,1974,1988,True,1974-1988
4,co8031861,1976,1976,True,1976
...,...,...,...,...,...
210492,co47145,<NA>,<NA>,True,Unknown
210493,co8538881,1960,1969,True,1960-1969
210494,co135801,1570,1700,True,1570-1700
210495,co8191358,1956,1956,True,1956
